In [1]:
from Lattes import Lattes
from bs4 import BeautifulSoup, Tag
import sys

# Dict appens even if key doesn't exists
https://stackoverflow.com/questions/12905999/how-to-create-key-or-append-an-element-to-key

## Using dict().setdefault():

dict_x = {}

...

dict_x.setdefault(key, []).append(value)
Cons: Inefficient creation of unused list()s.

dict.setdefault(key,[]).append(value)
help(dict.setdefault):

    setdefault(...)
        D.setdefault(k[,d]) -> D.get(k,d), also set D[k]=d if k not in D
        
## Using collections.defaultdict:

import collections
dict_x = collections.defaultdict(list)

...

dict_x[key].append(value)        

In [2]:
import collections 
lattes = collections.defaultdict(list)

In [3]:
clattes = Lattes()
clattes.read_zip_from_disk()
print (clattes.xml)

Arquivo compactado lido no disco.
XML atualizado a partir do Lattes compactado.
<?xml version="1.0" standalone="no"?><CURRICULO-VITAE SISTEMA-ORIGEM-XML="LATTES_OFFLINE" NUMERO-IDENTIFICADOR="7281587998425548" DATA-ATUALIZACAO="03022022" HORA-ATUALIZACAO="113653"><DADOS-GERAIS NOME-COMPLETO="Alberto de Campos e Silva" NOME-EM-CITACOES-BIBLIOGRAFICAS="Campos Silva, A" NACIONALIDADE="B" PAIS-DE-NASCIMENTO="Brasil" UF-NASCIMENTO="GO" CIDADE-NASCIMENTO="Goiânia" PERMISSAO-DE-DIVULGACAO="NAO" DATA-FALECIMENTO="" SIGLA-PAIS-NACIONALIDADE="BRA" PAIS-DE-NACIONALIDADE="Brasil" ORCID-ID="https://orcid.org/0000-0001-6506-1988"><RESUMO-CV TEXTO-RESUMO-CV-RH="Sou casado com uma mulher maravilhosa e tenho três lindos. Atualmente, sou Analista de C&amp;T no CNPq, lotado na COCEX/CGCEX/DEHS, analisando principalmente processos relacionados à Física e à Matemática. Tenho 46 anos, sou formado em física pela UnB, tendo dado aulas (de física) para o ensino fundamental por 2 anos (concursado GDF). Me mudei

In [4]:
soupa = BeautifulSoup(clattes.xml, "xml")
for child in soupa.children:
    print (child.name)

CURRICULO-VITAE


In [5]:
print (soupa.name)

[document]


In [6]:
xml = clattes.xml

In [7]:
from Indicadores import Indicadores
import pandas

In [8]:
import collections


In [9]:
    class I:
        indicadores = pandas.DataFrame()
    
    def indicadores_append (id, descricao, ano):
        if ano == None or ano == '': 
            ano = 0
        mydict = pandas.DataFrame({
            'Id': [id],
            'Tipo': [descricao],
            'Ano': [int(ano)]
        })
        I.indicadores = I.indicadores.append(mydict)

In [10]:
import os
from Lattes import Lattes
from Indicadores import Indicadores

lattes = collections.defaultdict(list)


pasta = r'C:\Users\silva\CNPq\Lattes\Downloads'
caminhos = [os.path.join(pasta, nome) for nome in os.listdir(pasta)]
arquivos = [arq for arq in caminhos if os.path.isfile(arq)]
zips = [arq for arq in arquivos if arq.lower().endswith(".zip")]
número = 0
for zip in zips:
    if os.path.basename(zip)[0:7]=='Lattes_':
        número += 1
        id = os.path.basename(zip)[7:-4]
        print(f'{número}: Recuperando dados do Currículo: {id}')
        lattes = Lattes()
        lattes.id = id
        lattes.read_zip_from_disk()
        indicadores = Indicadores(lattes.xml, id = id)
        indicadores.get_indicadores()    
    
    




def get_all(xml, nivel = 0):
    
    dado={}   
    if xml == None:
        return None
    if not (
            type(xml) == BeautifulSoup or
            type(xml) == Tag ):
        xml = BeautifulSoup(xml, "xml")
    nome_indicador = ''   
    if not xml.find('CURRICULO-VITAE') == None:
        xml = xml.find('CURRICULO-VITAE')

    
    
    parents = []
    for parent in xml.parents:
        parents.append(parent.name)
        nome_indicador = parent.name + ' - ' + nome_indicador
    nome_indicador += xml.name


    for (key, value) in xml.attrs.items():
            dado[key] = value
            
    # Verifica se os filhos não são detalhamentos do pai
    detalhamento = False
    for child in xml.children:
        child_name = child.name
        if (child_name[0:14] == 'DADOS-BASICOS' or
                child_name[0:13] == 'DETALHAMENTO-' or
                child_name == 'AUTORES' or
                child_name == "PALAVRAS-CHAVE" or
                child_name == "AREAS-DO-CONHECIMENTO" or
                child_name == "SETORES-DE-ATIVIDADE" or
                child_name == "INFORMACOES-ADICIONAIS" or
                child_name == 'INFORMACOES-ADICIONAIS' or
                child_name == 'INFORMACOES-ADICIONAIS-INSTITUICOES' or
                child_name == 'INFORMACOES-ADICIONAIS-CURSOS' 
        ):
            detalhamento = True
            continue
    
    # o que fazer se os filhos forem detalhamentos do pai
    if detalhamento:
        
        #1. Pegar autores, se existirem
        autores = []
        num_autores = 0
        posicao_autor = 0
        for d in xml.find_all('AUTORES'):
            dado = d.attrs
            num_autores += 1
            try:
                if d["NRO-ID-CNPQ"] == id:
                    posicao_autor = num_autores
            except:
                pass
            autores.append(dado)
        if len(autores) > 0:
            dado['POSICAO-AUTOR'] = posicao_autor
            dado['NUMERO-AUTORES'] = num_autores
            dado['AUTORES'] = autores
        
        #2 Pegar palavras-chave, se existirem.
        palavras_chaves = []
        for palavra in xml.find_all('PALAVRAS-CHAVE'):
            palavras_chaves.append(list(palavra.attrs.values()))
        if len(palavras_chaves) > 0:
            dado['PALAVRAS-CHAVES'] = palavras_chaves
            
        #3 Pegar áreas de atuação, se existirem
        areas = []
        for area in xml.find_all('AREAS-DO-CONHECIMENTO'):
            for a in area.children:
                area_ga = a.find('NOME-GRANDE-AREA-DO-CONHECIMENTO')
                area_a = a.find('NOME-DA-AREA-DO-CONHECIMENTO')
                area_sa = a.find('NOME-DA-SUB-AREA-DO-CONHECIMENTO')
                area_e = a.find('NOME-DA-ESPECIALIDADE')
                areas.append ((area_ga, area_a, area_sa, area_e))
        if len(areas) > 0:
            dado['AREAS-DO-CONHECIMENTO'] = areas
                     
        #4 Setores de Atividade
        setores = []
        for setor in xml.find_all('SETORES-DE-ATIVIDADE'):
            setores.append(list(setor.attrs.values()))
        if len(setores) > 0:
                 dado['SETORES-DE-ATIVIDADE'] = setores
                 
        #5 Informações adicionais
        informacoes = []
        for palavra in xml.find_all('INFORMACOES-ADICIONAIS'):
            informacoes.append(list(palavra.attrs.values()))
        if len(informacoes) > 0:
                 dado['INFORMACOES-ADICIONAIS'] = informacoes
                 
        #6 Pegar outras informações
        for child in xml.children:
            child_name = child.name
            if child_name in [
                    'AUTORES',
                    'PALAVRAS-CHAVE',
                    'AREAS-DO-CONHECIMENTO',
                    'SETORES-DE-ATIVIDADE',
                    'INFORMACOES-ADICIONAIS',
                    'INFORMACOES-ADICIONAIS-INSTITUICOES',
                    'INFORMACOES-ADICIONAIS-CURSOS',
                    'SISTEMA-ORIGEM-XML',
                    '[document] - '
                    ]:
                continue
            dado[child_name] = child.attrs
            for child2 in child.children:
                child_name = child2.name
                if child_name in [
                        'AUTORES',
                        'PALAVRAS-CHAVE',
                        'AREAS-DO-CONHECIMENTO',
                        'SETORES-DE-ATIVIDADE',
                        'INFORMACOES-ADICIONAIS',
                        ]:
                    continue
                dado[child_name] = child2.attrs
                
    else:
        # se não for detalhamento, continua a importação
        dado['ID'] = id
        print(f'inserinfo indicador { nome_indicador}')
        if not dado.get('ANO') == None:
            ano = dado.get('ANO')
        elif not dado.get('ANO-DE-CONCLUSAO') == None:
            ano = dado.get('ANO-DE-CONCLUSAO')
        elif not dado.get('ANO-DA-OBRA-DE-REFERENCIA') == None:
            ano = dado.get('ANO-DA-OBRA-DE-REFERENCIA')
        elif not dado.get('ANO-DE-OBTENCAO-DO-TITULO') ==None:
            ano = dado.get('ANO-DE-OBTENCAO-DO-TITULO')
        elif not dado.get('ANO-FIM') == None:
            ano = dado.get('ANO-FIM')
        elif not dado.get('ANO-DE-REALIZACAO') == None:
            ano = dado.get('ANO-DE-REALIZACAO')
        elif not dado.get('ANO-DO-ARTIGO') == None:
            ano = dado.get('ANO-DO-ARTIGO')
        elif not dado.get('ANO-DO-TEXTO') == None:
            ano = dado.get('ANO-DO-TEXTO')
        elif not dado.get('ANO-DESENVOLVIMENTO') == None:
            ano = dado.get('ANO-DESENVOLVIMENTO')
        elif not dado.get('ANO-SOLICITACAO') == None:
            ano = dado.get('ANO-SOLICITACAO')
        elif not dado.get('ANO-DA-OBRA') == None:
            ano = dado.get('ANO-DA-OBRA') 
        else:
            ano = None
        if not ano==None:
            dado['ANO'] = ano
        lattes[nome_indicador].append(dado)
        indicadores_append(id, nome_indicador, ano)

        for child in xml.children:
            get_all(child, nivel + 1)
                
            
        
get_all(soupa)
        
print(lattes)
            

inserinfo indicador [document] - CURRICULO-VITAE
inserinfo indicador [document] - CURRICULO-VITAE - DADOS-GERAIS
inserinfo indicador [document] - CURRICULO-VITAE - DADOS-GERAIS - RESUMO-CV
inserinfo indicador [document] - CURRICULO-VITAE - DADOS-GERAIS - ENDERECO
inserinfo indicador [document] - CURRICULO-VITAE - DADOS-GERAIS - ENDERECO - ENDERECO-PROFISSIONAL
inserinfo indicador [document] - CURRICULO-VITAE - DADOS-GERAIS - FORMACAO-ACADEMICA-TITULACAO
inserinfo indicador [document] - CURRICULO-VITAE - DADOS-GERAIS - FORMACAO-ACADEMICA-TITULACAO - GRADUACAO
inserinfo indicador [document] - CURRICULO-VITAE - DADOS-GERAIS - FORMACAO-ACADEMICA-TITULACAO - ENSINO-FUNDAMENTAL-PRIMEIRO-GRAU
inserinfo indicador [document] - CURRICULO-VITAE - DADOS-GERAIS - FORMACAO-ACADEMICA-TITULACAO - ENSINO-MEDIO-SEGUNDO-GRAU
inserinfo indicador [document] - CURRICULO-VITAE - DADOS-GERAIS - ATUACOES-PROFISSIONAIS
inserinfo indicador [document] - CURRICULO-VITAE - DADOS-GERAIS - ATUACOES-PROFISSIONAIS - AT

        
def oi():        
    
                    
                
        if not dado.get('NATUREZA') == None:
            nome_indicador[nivel] += ' - ' + dado['NATUREZA']
            
        
        for child in xml.children:
            child_name = child.name
            if not (child_name[0:14] == 'DADOS-BASICOS' or
                    child_name[0:13] == 'DETALHAMENTO-' or
                    child_name == 'AUTORES' or
                    child_name == "PALAVRAS-CHAVE" or
                    child_name == "AREAS-DO-CONHECIMENTO" or
                    child_name == "SETORES-DE-ATIVIDADE" or
                    child_name == "INFORMACOES-ADICIONAIS"
            ):
                nome_indicador.append(xml.name)
                print('Importando: ', nome_indicador)
                lattes[nome_indicador[nivel]] = xml.attrs
                nivel += 1
                for child in xml:
                    get_children(child, nivel, nome_indicador)
        else:
            dado = {}

In [11]:
I.indicadores.Tipo.to_csv('teste.csv')

In [12]:
print(I.indicadores.Ano)

0       0
0       0
0       0
0       0
0       0
0       0
0    2000
0    1990
0    1993
0       0
0       0
0    2002
0       0
0    2012
0       0
0       0
0       0
0       0
0       0
0       0
0       0
0       0
0    2011
0       0
0       0
0       0
0       0
0       0
0       0
0       0
0       0
0       0
0       0
0       0
0       0
0       0
0       0
Name: Ano, dtype: int64


In [13]:
for [key, value] in lattes.items():
    print(f'Key: {key}, Value:{value}')

Key: [document] - CURRICULO-VITAE, Value:[{'SISTEMA-ORIGEM-XML': 'LATTES_OFFLINE', 'NUMERO-IDENTIFICADOR': '7281587998425548', 'DATA-ATUALIZACAO': '03022022', 'HORA-ATUALIZACAO': '113653', 'ID': '7281587998425548'}]
Key: [document] - CURRICULO-VITAE - DADOS-GERAIS, Value:[{'NOME-COMPLETO': 'Alberto de Campos e Silva', 'NOME-EM-CITACOES-BIBLIOGRAFICAS': 'Campos Silva, A', 'NACIONALIDADE': 'B', 'PAIS-DE-NASCIMENTO': 'Brasil', 'UF-NASCIMENTO': 'GO', 'CIDADE-NASCIMENTO': 'Goiânia', 'PERMISSAO-DE-DIVULGACAO': 'NAO', 'DATA-FALECIMENTO': '', 'SIGLA-PAIS-NACIONALIDADE': 'BRA', 'PAIS-DE-NACIONALIDADE': 'Brasil', 'ORCID-ID': 'https://orcid.org/0000-0001-6506-1988', 'ID': '7281587998425548'}]
Key: [document] - CURRICULO-VITAE - DADOS-GERAIS - RESUMO-CV, Value:[{'TEXTO-RESUMO-CV-RH': 'Sou casado com uma mulher maravilhosa e tenho três lindos. Atualmente, sou Analista de C&T no CNPq, lotado na COCEX/CGCEX/DEHS, analisando principalmente processos relacionados à Física e à Matemática. Tenho 46 anos,